# 설치 안된 모듈 설치

In [23]:
# conda install selenium
# pip install fake_useragent
# pip install html_table_parser
# conda install sqlalchemy
# conda install numpy
# conda install joblib
# pip install beautifulsoup4
# pip install --upgrade html5lib

# 패키지 import

In [11]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import MySQLdb
import numpy as np
import warnings
import datetime as dt
warnings.filterwarnings(action='ignore')

In [12]:
import telegram   #텔레그램 모듈을 가져옵니다.

my_token = '1933232688:AAHSeyS3qAtZnl83_sWEWclAmSd2LOod_ZI'   #토큰을 변수에 저장합니다.

bot = telegram.Bot(token = my_token)   #bot을 선언합니다.

chat_id = '1906048846' #chat_id 선언


# 배당률 불러오기

## 일반 승무패

* 8대리그 
 - 8910 K리그
 - 94 프리미어리그
 - 123 분데스리가
 - 207 라리가
 - 99 리그앙
 - 199 세리에
 - 895 J리그
 - 849 CSL

In [13]:
d = dt.datetime.now()
bot.sendMessage(chat_id, "[START] 배당률 작업 시작. \n {}".format(d))

In [23]:

#L스포츠 배당률 데이터 불러오기
db_connection_str = '-'
db_connection = create_engine(db_connection_str)

#승무패 배당률. market_id = 1_1, 리그 코드 확인

bet1 = pd.read_sql('''
                    SELECT fixture.league_id, fixture.start_date, home.name as home_name, away.name as away_name, bet.provider_id, provider.name, market_id, bet.name as wdl, bet.price, status
                    FROM lsports_shadow.fixture
                    JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
                    JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
                    JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
                    JOIN lsports.provider on bet.provider_id = provider.id
                    WHERE (league_id = 8910 AND league_id = 895 or league_id = 849 or league_id = 94 or league_id = 123 or league_id = 207  
                    or league_id = 99 or league_id = 199 ) AND bet.market_id = '1_1'AND provider_id = 8

                    ''', \
                     con=db_connection)

# b_dfb1 = pd.read_sql('''
#                     SELECT fixture.league_id, fixture.start_date, home.name as home_name, away.name as away_name, bet.provider_id, provider.name, market_id, bet.name as wdl, bet.price, status
#                     FROM lsports_shadow.fixture
#                     JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
#                     JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
#                     JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
#                     JOIN lsports.provider on bet.provider_id = provider.id
#                     WHERE fixture.sport_id = 16 AND fixture.league_id = 349 AND bet.market_id = '16_1'
#                     ''', \
#                          con=db_connection)

#데이터프레임 형식으로 만들기

def bet_norm(data):

    data = pd.DataFrame(data.groupby(['start_date','league_id', 'home_name', 'away_name', 'wdl','status']).mean()['price'].reset_index())

    temp = data.values.tolist()

    dataTemp = []
    row = {}
    for i in range(0,len(temp)):
        if i % 3 == 0:
            row = {}
            row['date'] = temp[i][0]
            row['league_id'] = temp[i][1]
            row['home'] = temp[i][2]
            row['away'] = temp[i][3]
            row['status'] = temp[i][5]
            row['home_odd'] = temp[i][6]
        elif i % 3 == 1:
            row['away_odd'] = temp[i][6]
        elif i % 3 == 2:
            row['draw_odd'] = temp[i][6]
            dataTemp.append(row)

    bet_1 = pd.DataFrame(dataTemp)
    
    
    #배당률 이상치 보정

#     for i in range(0,len(bet_1)):
#         if bet_1['home_odd'][i] > 15:
#             bet_1['home_odd'][i] = 15
#         elif bet_1['draw_odd'][i] > 15:
#             bet_1['draw_odd'][i] = 15
#         elif bet_1['away_odd'][i] > 15:
#             bet_1['away_odd'][i] = 15

    import datetime as dt
    bet_1['date'] = bet_1['date'].dt.date

    bet_1 = bet_1.reset_index()
    bet_1 = bet_1.drop('index',axis=1)
    
    bet_1.insert(2,'league','')
    
    for i in range(0,len(bet_1)):
        if bet_1['league_id'][i] == 8910:
            bet_1['league'][i] = 'K League 1'
        elif bet_1['league_id'][i] == 895:
            bet_1['league'][i] = 'J1 League'
        elif bet_1['league_id'][i] == 849:
            bet_1['league'][i] = 'CSL'
        elif bet_1['league_id'][i] == 94:
            bet_1['league'][i] = 'England Premire League'
        elif bet_1['league_id'][i] == 123:
            bet_1['league'][i] = 'Bundesliga'
        elif bet_1['league_id'][i] == 207:
            bet_1['league'][i] = 'Primera Division'
        elif bet_1['league_id'][i] == 99:
            bet_1['league'][i] = 'League 1'
        elif bet_1['league_id'][i] == 199:
            bet_1['league'][i] = 'Serie A'
    
    bet_1.dropna().reset_index().drop('index',axis=1)
    
    return bet_1


norm = bet_norm(bet1)
# norm2 = bet_norm(b_dfb1)
norm

,date,league_id,league,home,away,status,home_odd,away_odd,draw_odd
0,2021-05-21,207,Primera Division,Levante,Cadiz,3,1.68,4.80,4.2
1,2021-05-22,123,Bundesliga,Bayern Munich,Augsburg,3,1.13,16.00,10.5
2,2021-05-22,123,Bundesliga,Borussia Dortmund,Bayer Leverkusen,3,1.56,5.10,4.9
3,2021-05-22,123,Bundesliga,Cologne,Schalke,3,1.37,6.80,5.6
4,2021-05-22,123,Bundesliga,Eintracht Frankfurt,SC Freiburg,3,2.14,3.00,4.2
...,...,...,...,...,...,...,...,...,...
411,2021-10-04,207,Primera Division,Villarreal,Real Betis,0,2.01,3.70,3.7
412,2021-10-04,123,Bundesliga,Arminia Bielefeld,Bayer Leverkusen,0,5.00,1.70,4.0
413,2021-10-04,99,League 1,St Etienne,Lyon,0,4.10,1.80,4.1
414,2021-10-04,199,Serie A,Atalanta,AC Milan,0,2.12,3.40,3.8


In [39]:
db_connection_str = '-'
db_connection = create_engine(db_connection_str)

bet2 = pd.read_sql('''
                    SELECT fixture.league_id, fixture.start_date, home.name as home_name, away.name as away_name, bet.provider_id, provider.name, market_id, bet.name as wdl, bet.price, bet.base_line, status
                    FROM lsports_shadow.fixture
                    JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
                    JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
                    JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
                    JOIN lsports.provider on bet.provider_id = provider.id
                    WHERE (league_id = 8910 AND league_id = 895 or league_id = 94 or league_id = 123 or league_id = 207  
                    or league_id = 99 or league_id = 199 ) AND bet.market_id = '1_2'AND provider_id = 8

                    ''', \
                     con=db_connection)

# b_dfb2 = pd.read_sql('''
#                     SELECT fixture.league_id, home.name as home_name, away.name as away_name, bet.provider_id, provider.name as provider_name,market_id, bet.name, bet.price , start_date, base_line, status
#                     FROM lsports_shadow.fixture
#                     JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
#                     JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
#                     JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
#                     JOIN lsports.provider on bet.provider_id = provider.id
#                     WHERE fixture.sport_id = 16 AND fixture.league_id = 349 AND bet.market_id = '16_2'

#                     ''', \
#                          con=db_connection)

# bet2
def bet_handi(data):
#     data = pd.DataFrame(data.groupby(['start_date','league_id', 'home_name', 'away_name','name', 'base_line','status']).mean()['price'].reset_index())
    temp = data.values.tolist()
#     return temp

    dataTemp = []
    row = {}

    for i in range(0,len(temp)):
        if i % 2 == 0:
            row = {}
            row['date'] = temp[i][1]
            row['league_id'] = temp[i][0]
            row['home'] = temp[i][2]
            row['away'] = temp[i][3]
            row['provider'] = temp[i][5]
            row['base_line'] = temp[i][9]
            row['status'] = temp[i][10]
            row['home_odd'] = temp[i][8]
        elif i % 2 == 1:
            row['away_odd'] = temp[i][8]
            dataTemp.append(row)

    bet_2 = pd.DataFrame(dataTemp)
    
# return bet_2

    import datetime as dt
    bet_2['date'] = bet_2['date'].dt.date

    bet_2 = bet_2.reset_index()
    bet_2 = bet_2.drop('index',axis=1)

    bet_2 = bet_2.sort_values(by=['date','home'])
    bet_2 = bet_2.reset_index()
    bet_2 = bet_2.drop(['index'],axis=1)
    

    bet_2.insert(2,'league','')
    
    for i in range(0,len(bet_2)):
        if bet_2['league_id'][i] == 8910:
            bet_2['league'][i] = 'K League 1'
        elif bet_2['league_id'][i] == 895:
            bet_2['league'][i] = 'J1 League'
        elif bet_2['league_id'][i] == 849:
            bet_2['league'][i] = 'CSL'
        elif bet_2['league_id'][i] == 94:
            bet_2['league'][i] = 'England Premire League'
        elif bet_2['league_id'][i] == 123:
            bet_2['league'][i] = 'Bundesliga'
        elif bet_2['league_id'][i] == 207:
            bet_2['league'][i] = 'Primera Division'
        elif bet_2['league_id'][i] == 99:
            bet_2['league'][i] = 'League 1'
        elif bet_2['league_id'][i] == 199:
            bet_2['league'][i] = 'Serie A'
            
            
    return bet_2

handi = bet_handi(bet2)
handi
# handi2 = bet_handi(b_dfb2)


,date,league_id,league,home,away,provider,base_line,status,home_odd,away_odd
0,2021-05-21,207,Primera Division,Levante,Cadiz,Bet365,-0.50,3,1.91,2.05
1,2021-05-22,123,Bundesliga,Bayern Munich,Augsburg,Bet365,-2.75,3,2.08,1.88
2,2021-05-22,123,Bundesliga,Borussia Dortmund,Bayer Leverkusen,Bet365,-1.25,3,1.89,2.06
3,2021-05-22,199,Serie A,Cagliari,Genoa,Bet365,-0.25,3,2.09,1.86
4,2021-05-22,207,Primera Division,Celta Vigo,Real Betis,Bet365,0.25,3,2.01,1.95
...,...,...,...,...,...,...,...,...,...,...
342,2021-10-04,99,League 1,Lille,Marseille,Bet365,0.00,0,2.14,1.81
343,2021-10-04,94,England Premire League,Liverpool,Man City,Bet365,0.25,0,2.14,1.84
344,2021-10-04,199,Serie A,Roma,Empoli,Bet365,-1.25,0,1.88,2.09
345,2021-10-04,99,League 1,St Etienne,Lyon,Bet365,0.75,0,2.11,1.84


In [41]:
db_connection_str = '-'
db_connection = create_engine(db_connection_str)

bet3 = pd.read_sql('''
                    SELECT fixture.league_id, fixture.start_date, home.name as home_name, away.name as away_name, 
                    bet.provider_id, provider.name, market_id, bet.name as wdl, bet.price, bet.base_line, status
                    FROM lsports_shadow.fixture
                    JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
                    JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
                    JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
                    JOIN lsports.provider on bet.provider_id = provider.id
                    WHERE (league_id = 8910 AND league_id = 895 or league_id = 849 or league_id = 94 or league_id = 123 or league_id = 207  
                    or league_id = 99 or league_id = 199 ) AND bet.market_id = '1_3'AND provider_id = 8

                    ''', \
                     con=db_connection)

# b_dfb3 = pd.read_sql('''
#                     SELECT fixture.league_id, home.name as home_name, away.name as away_name, bet.provider_id, provider.name as provider_name,market_id, bet.name, bet.price , start_date, base_line, status
# FROM lsports_shadow.fixture
# JOIN lsports_shadow.participant AS home on fixture.home_id = home.id
# JOIN lsports_shadow.participant AS away on fixture.away_id = away.id
# JOIN lsports_shadow.bet on fixture.id = bet.fixture_id
# JOIN lsports.provider on bet.provider_id = provider.id
# WHERE fixture.sport_id = 16 AND fixture.league_id = 349 AND bet.market_id = '16_3'

#                 ''', \
#                      con=db_connection)

def bet_unov(data):
    data.columns = ['league_id','start_date','home_name','away_name','provider_id','provider_name','market_id','wdl','price','base_line','status']

# 배당률 이상치 보정
#     for i in range(0,len(data)):
#         if data['price'][i] > 15:
#             data['price'][i] = 15

#     data = pd.DataFrame(data.groupby(['start_date','league_id', 'home_name', 'away_name','provider_name', 'base_line','status']).mean()['price'].reset_index())
    temp = data.values.tolist()
#     return temp

    dataTemp = []
    row = {}

    for i in range(0,len(temp)):
        if i % 2 == 0:
            row = {}
            row['date'] = temp[i][1]
            row['league_id'] = temp[i][0]
            row['home'] = temp[i][2]
            row['away'] = temp[i][3]
            row['provider'] = temp[i][5]
            row['base_line'] = temp[i][9]
            row['status'] = temp[i][10]
            row['over_odd'] = temp[i][8]
        elif i % 2 == 1:
            row['under_odd'] = temp[i][8]
            dataTemp.append(row)

    bet_3 = pd.DataFrame(dataTemp)

    bet_3['date'] = bet_3['date'].dt.date

    bet_3 = bet_3.reset_index()
    bet_3 = bet_3.drop('index',axis=1)

    bet_3 = bet_3.sort_values(by=['date','home'])
    bet_3 = bet_3.reset_index()
    bet_3 = bet_3.drop(['index'],axis=1)
    
    bet_3.insert(2,'league','')
    
    for i in range(0,len(bet_3)):
        if bet_3['league_id'][i] == 8910:
            bet_3['league'][i] = 'K League 1'
        elif bet_3['league_id'][i] == 895:
            bet_3['league'][i] = 'J1 League'
        elif bet_3['league_id'][i] == 849:
            bet_3['league'][i] = 'CSL'
        elif bet_3['league_id'][i] == 94:
            bet_3['league'][i] = 'England Premire League'
        elif bet_3['league_id'][i] == 123:
            bet_3['league'][i] = 'Bundesliga'
        elif bet_3['league_id'][i] == 207:
            bet_3['league'][i] = 'Primera Division'
        elif bet_3['league_id'][i] == 99:
            bet_3['league'][i] = 'League 1'
        elif bet_3['league_id'][i] == 199:
            bet_3['league'][i] = 'Serie A'
    
    return bet_3

unov = bet_unov(bet3)
unov
# # unov2 = bet_unov(b_dfb3)


,date,league_id,league,home,away,provider,base_line,status,over_odd,under_odd
0,2021-05-21,207,Primera Division,Levante,Cadiz,Bet365,3.00,3,2.03,1.91
1,2021-05-22,123,Bundesliga,Bayern Munich,Augsburg,Bet365,4.50,3,1.88,2.06
2,2021-05-22,123,Bundesliga,Borussia Dortmund,Bayer Leverkusen,Bet365,3.75,3,1.94,2.00
3,2021-05-22,199,Serie A,Cagliari,Genoa,Bet365,3.00,3,1.90,2.04
4,2021-05-22,207,Primera Division,Celta Vigo,Real Betis,Bet365,3.00,3,2.00,1.94
...,...,...,...,...,...,...,...,...,...,...
412,2021-10-04,99,League 1,Lille,Marseille,Bet365,2.25,0,1.89,2.05
413,2021-10-04,94,England Premire League,Liverpool,Man City,Bet365,2.75,0,1.93,2.01
414,2021-10-04,199,Serie A,Roma,Empoli,Bet365,3.50,0,2.05,1.89
415,2021-10-04,99,League 1,St Etienne,Lyon,Bet365,3.00,0,1.98,1.96


In [7]:
import os
import re
import dateutil
from sklearn.preprocessing import LabelEncoder
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from fake_useragent import UserAgent
from html_table_parser import parser_functions as parser


def insert_db1(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd', con=engine, if_exists='replace', index=False)

def insert_db2(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd_handicap', con=engine, if_exists='replace', index=False)

def insert_db3(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd_uo', con=engine, if_exists='replace', index=False)


In [8]:

insert_db1(norm)

insert_db2(handi)

insert_db3(unov)


In [9]:
def insert_db1_b(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd', con=engine, if_exists='replace', index=False)

def insert_db2_b(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd_handicap', con=engine, if_exists='replace', index=False)

def insert_db3_b(df):

    engine = create_engine("-", encoding='utf-8')
    conn = engine.connect()

    return df.to_sql(name='odd_uo', con=engine, if_exists='replace', index=False)

In [10]:
insert_db1_b(norm2)
insert_db2_b(handi2)
insert_db3_b(unov2)


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '121.135.204.178' (timed out)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
d = dt.datetime.now()
bot.sendMessage(chat_id = chat_id, text="[SUCCESS] 배당률 작업 완료. \n {}".format(d))